# FMLIKH Model: Stress Relaxation

## Protocol-Specific Context (Multi-Mode)

**FMLIKH relaxation** combines **multi-exponential decay** with **power-law tails**:

1. **Short times**: Sum of stretched exponentials from each mode
2. **Intermediate times**: Crossover between modes as fast modes relax first
3. **Long times**: Slowest mode's power-law tail $\sim t^{-\alpha_{slow}}$ dominates

**Why this matters**: Real materials often show **non-single-exponential relaxation** that's not purely power-law either. FMLIKH captures this via superposition: $\sigma(t) = \sum_i w_i \sigma_i(t)$ where each $\sigma_i \sim E_{\alpha_i}(-(t/\tau_i)^{\alpha_i})$.

> **Physical insight**: Fast modes relax away quickly, leaving slow modes to dominate long-time response. If slow modes have low $\alpha_i$ (strong memory), the tail is very persistent — material "remembers" initial strain for extended times.

> **Handbook:** See [FMLIKH Relaxation](../../docs/source/models/fikh/fmlikh.rst) for multi-mode Mittag-Leffler superposition.

## Learning Objectives

1. Simulate FMLIKH relaxation with N=3 modes
2. Observe multi-exponential → power-law crossover
3. Analyze mode-by-mode stress decay
4. Compare FMLIKH vs single-mode FIKH relaxation spectra
5. Extract effective $\alpha$ from long-time slope

## Prerequisites

- NB07: FMLIKH flow curve
- NB03: FIKH relaxation (single-mode baseline)

**Estimated Time:** 3-5 minutes (fast), 12-15 minutes (full)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_fikh_parameters, generate_synthetic_relaxation, save_fikh_results,
    set_model_parameters, print_convergence_summary, compute_fit_quality,
    get_fmlikh_param_names,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit,
    display_arviz_diagnostics,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# ============================================================
# FAST_MODE Configuration
# ============================================================
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

if FAST_MODE:
    print("FAST_MODE: reduced data/samples for quick validation")
    N_DATA_POINTS = 50
    NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 50, 100, 1
else:
    print("FULL mode: complete Bayesian inference")
    N_DATA_POINTS = 200
    NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

## 2. Load Calibrated Parameters

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

try:
    params = load_fikh_parameters("fmlikh", "flow_curve")
    set_model_parameters(model, params)
    print("Loaded parameters from NB07")
except FileNotFoundError:
    print("Using default parameters")

## 3. Generate Synthetic Data

In [ ]:
SIGMA_0 = 100.0
time_data, stress_data = generate_synthetic_relaxation(
    model, sigma_0=SIGMA_0, t_end=100.0, n_points=N_DATA_POINTS, noise_level=0.03, seed=42
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(time_data, stress_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title("Synthetic FMLIKH Relaxation Data")
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
model_fit = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.5)

t0 = time.time()
model_fit.fit(time_data, stress_data, test_mode="relaxation", sigma_0=SIGMA_0, method='scipy')
print(f"NLSQ time: {time.time() - t0:.2f} s")

stress_pred = model_fit.predict(time_data, test_mode="relaxation", sigma_0=SIGMA_0)
metrics = compute_fit_quality(stress_data, stress_pred)
print(f"R² = {metrics['R2']:.6f}")

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)

fig, ax = plot_nlsq_fit(
    time_data, stress_data, model_fit, test_mode="relaxation",
    param_names=param_names, log_scale=True,
    xlabel="Time [s]", ylabel="Stress [Pa]",
    title=f"FMLIKH Relaxation Fit (R$^2$ = {metrics['R2']:.5f})",
    sigma_0=SIGMA_0,
)
display(fig)
plt.close(fig)

## 5. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model_fit.parameters.get_value(n) for n in param_names if n in model_fit.parameters}

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
print(f"  FAST_MODE={FAST_MODE}, N_MODES={N_MODES}, data points={len(time_data)}")

t0 = time.time()
result = model_fit.fit_bayesian(
    time_data, stress_data, test_mode="relaxation", sigma_0=SIGMA_0,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure"]
print_convergence_summary(result, key_params)

print("\n### Bayesian Diagnostics")
print("| Metric | Target | Interpretation |")
print("|--------|--------|----------------|")
print("| **R-hat** | < 1.01 | Convergence indicator |")
print("| **ESS** | > 400 | Sample quality |")
print("| **Divergences** | < 1% | Sampling reliability |")

display_arviz_diagnostics(result, key_params, fast_mode=FAST_MODE)

### Convergence Diagnostics

**Bayesian Diagnostic Targets:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Convergence indicator |
| **ESS** | > 400 | Sample quality |
| **Divergences** | < 1% | Sampling reliability |

## 6. Save Results

In [ ]:
save_fikh_results(model_fit, result, "fmlikh", "relaxation", param_names)

## Key Takeaways

1. **FMLIKH relaxation: multi-exponential → power-law crossover**
2. **Fast modes relax first; slowest mode dominates long-time tail**
3. **Long-time slope reveals $\alpha$ of slowest mode**
4. **Broad relaxation spectra** better captured by multi-mode
5. **Superposition of Mittag-Leffler functions** gives rich dynamics
6. **Residual analysis** validates mode decomposition and power-law tails

---

## Further Reading

- **[FMLIKH Relaxation Spectra](../../docs/source/models/fikh/fmlikh.rst#stretched-exponential-vs-fractional-modes)**: Mode superposition and asymptotic analysis
- **[Per-Mode Timescales](../../docs/source/models/fikh/fmlikh.rst#per-mode-timescales)**: Logarithmic distribution recommendations

### Key References

1. Mainardi, F. (2010). *Fractional Calculus and Waves in Linear Viscoelasticity*. Imperial College Press.
2. Wei, Y. et al. (2018). "A multimode structural kinetics constitutive equation." *J. Rheol.*, 62(1), 321-342.

### Next Steps

**Next**: NB10 (Creep) — distributed delay times from hierarchical structure